In [7]:
from typing import List
import torch
import torch._dynamo as dynamo

In [2]:
print(torch.__version__)

2.0.0.dev20221210


In [1]:
dynamo.config.suppress_errors = True

def my_compiler(gm: torch.fx.GraphModule, example_inputs: List[torch.Tensor]):
    print("my_compiler() called with FX graph:")
    gm.graph.print_tabular()
    return gm.forward  # return a python callable

@dynamo.optimize(my_compiler)
def foo(a, b):
    x = a / (torch.abs(a) + 1)
    if b.sum() < 0:
        b = b* -1
    return x * b

foo(torch.randn(10), torch.randn(10))

my_compiler() called with FX graph:
opcode         name     target                                               args              kwargs
-  -  ------  -  --
placeholder    a        a                                                    ()                {}
placeholder    b        b                                                    ()                {}
call_function  abs_1    <built-in method abs of type object at 0x111599f20>  (a,)              {}
call_function  add      <built-in function add>                              (abs_1, 1)        {}
call_function  truediv  <built-in function truediv>                          (a, add)          {}
call_method    sum_1    sum                                                  (b,)              {}
call_function  lt       <built-in function lt>                               (sum_1, 0)        {}
output         output   output                                               ((truediv, lt),)  {}
my_compiler() called with FX graph:
opcode         name   

tensor([-0.0283,  0.5387, -0.3300,  0.3821,  0.0061, -0.0321,  0.5912,  0.0104,
         0.0019,  0.0826])